# Setup

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')

In [ ]:
!pip install 'crewai[tools]' -q
!pip install 'langchain-openai' -q

In [ ]:
# Import libraries
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
import os
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

In [ ]:
# Set the API Keys as environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
# Define the LLM
llm = ChatOpenAI(model_name = "gpt-4o", temperature = 0.5)

# Get the data

* Interviewer
* Company
* Job Position
* Job Description


In [ ]:
# Store the data
interviewer = input('Enter the name of the interviewer: ')
company = input('Enter the name of the company: ')
job_position = input('Enter the job position: ')
job_description = input('Enter the job description: ')

# Interviewer AI Agent

In [ ]:
# Create the interver AI agent
interviewer_agent = Agent(
    role = f" You are {interviewer}, employed at {company}, and hiring for job position {job_position}",
    goal = f" you help the user user prepare for the job interview for job {job_position} with the following description: {job_description}",
    backstory = """
    Experience
    BEAT81
    Analytics Engineer

    BEAT81 · Full-timeBEAT81 · Full-time
    Apr 2024 - Present · 11 mos
    Berlin, Germany
    diconium
    Full-time · 4 yrs 2 mosFull-time · 4 yrs 2 mos

    Senior Cloud Architect
    Apr 2022 - Mar 2024 · 2 yrs
    Microsoft Azure, DBT and +4 skills

    Data Engineer
    Jan 2021 - Mar 2022 · 1 yr 3 mosJan 2021 to Mar 2022 · 1 yr 3 mos
    DBT, AWS and +4 skills

    Data Scientist
    Feb 2020 - Dec 2020 · 11 mos
    SQL and Python
    4flow logo

    Data Scientist
    4flow · Full-time4flow · Full-time
    Jun 2019 - Dec 2019 · 7 mosJun 2019 to Dec 2019 · 7 mos
    SQL and Python
    DCMN
    Full-time · 3 yrs 3 mos

    Data Scientist
    Apr 2017 - Apr 2019 · 2 yrs 1 moApr 2017 to Apr 2019 · 2 yrs 1 mo
    SQL and Python
    Data Analyst / Business Intelligence Manager

    Feb 2016 - Mar 2017 · 1 yr 2 mosFeb 2016 to Mar 2017 · 1 yr 2 mos
    SQL and Python
    iversity logo

    Data Analyst
    iversity · Full-timeiversity · Full-time
    Dec 2014 - Dec 2015 · 1 yr 1 moDec 2014 to Dec 2015 · 1 yr 1 mo
    SQL and R

    Education
    Technische Universität Berlin
    Master of Science - MS, Computational Neuroscience
    Sep 2009 - Aug 2013

    Universität Osnabrück
    Bachelor of Science - BS, Cognitive ScienceBachelor of Science - BS, Cognitive Science
    """,
    llm = llm
)

In [ ]:
# Define the interview prep task
interview_prep_task = Task(
    description = f"""
    Interview the user for the job {job_position} with the following description: {job_description}
    """,
    expected_output = f"""
    Ask only one question to the user that is relevant for the job {job_position}
    """,
    agent = interviewer_agent,
    human_input = True
)

# AI Coach

In [ ]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position} by grading the relevance of the answer",
    backstory = "You are an expert in job interviews",
    llm = llm
)

In [ ]:
# Define the Coaching Task
coaching_task = Task(
    description = f""" Give feedback to the user on its answer """,
    expected_output = f""" Give feedback to the user on its answer with bullet points on what was good, bad, and how to take it to the next level""",
    agent = coach_agent,
    context = [interview_prep_task],
)

# AI Crew

In [ ]:
# Create an empty list for questions
interview_questions = []

# # Store each question
# interview_questions.append(result.tasks_output[0].raw)
# interview_content = ' '.join(interview_questions)
# string_source = StringKnowledgeSource(content = interview_content)

In [ ]:
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, coaching_task],
    verbose = True,
    process = Process.sequential,
    memory = True,
    knowledge_sources = [string_source],
)
result = crew.kickoff()

# Store each question
interview_questions.append(result.tasks_output[0].raw)
interview_content = ' '.join(interview_questions)
string_source = StringKnowledgeSource(content = interview_content)

In [ ]:
# Check output
display(Markdown(result.tasks_output[1].raw))